In [32]:
LOG_DIR = "../log_junyi_testing_2/"

# Fetch Inervals from log file

In [33]:
import pandas as pd
import time
import datetime

def getIntervalTemplate(outputFile):
    # Get intermediate file
    import subprocess
    bashCommand = '''echo "time" > interval_intermediate.csv && cat ../log_junyi_testing_2/keyStroke.log | grep "\-\-\-5min" | grep -o '[0-9]*-[0-9]*-[0-9]*-[0-9]*:[0-9]*:[0-9]*' >> interval_intermediate.csv'''
    subprocess.check_output(bashCommand, shell=True)


    raw_time = pd.read_csv("interval_intermediate.csv")
    raw_time = pd.DataFrame(raw_time)

    all_time_stamp = []
    dateTime = str(raw_time.time[0])
    unixTime = time.mktime(datetime.datetime.strptime(dateTime, "%Y-%m-%d-%H:%M:%S").timetuple())
    for i in range(1,len(raw_time.time)):
        new_unixTime = time.mktime(datetime.datetime.strptime(str(raw_time.time[i]), "%Y-%m-%d-%H:%M:%S").timetuple())      
        all_time_stamp.append(datetime.datetime.fromtimestamp(unixTime).strftime('%Y-%m-%d-%H:%M:%S')+"--"+datetime.datetime.fromtimestamp(new_unixTime).strftime('%Y-%m-%d-%H:%M:%S'))
        unixTime = new_unixTime
    df = pd.DataFrame(all_time_stamp, columns=['time_intervals'])
    df.to_csv(outputFile, index=False, sep=' ')
    return df
    
getIntervalTemplate("interval_template.csv")[:5]

,time_intervals
0,2018-12-03-13:10:53--2018-12-03-13:15:54
1,2018-12-03-13:15:54--2018-12-03-13:20:54
2,2018-12-03-13:20:54--2018-12-03-13:25:55
3,2018-12-03-13:25:55--2018-12-03-13:30:55
4,2018-12-03-13:30:55--2018-12-03-13:35:55


# Fetch number of windows being touched in 5 mins

In [34]:
def windowTouchesCount():
    count = 0
    start = True
    windowContent = [line.strip() for line in open(LOG_DIR + "windowAudit.log", 'r')]
    windowCount_feature = []
    for line in windowContent:
        if line != "":
            if "---5min Window Audit milestone" in line:
                if start:
                    start = False
                else:
                    windowCount_feature.append(count)
                    count = 0 
            else:
                count += 1
    df = pd.DataFrame(windowCount_feature, columns=['touched_windows_count'])
    df.to_csv("windows_touched_feature.csv", index=False, sep=' ')
    return df

len(windowTouchesCount())

42

# Fetch Chrome, WeChat, Terminal lasting time in 5 mins (write in class)

In [35]:
class Interval(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end 
        self.lasting = 0

def unixTime_intervals(element):
    fromTime, toTime = element.split("--")
    # Conver to unix time
    fromTime = time.mktime(datetime.datetime.strptime(fromTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
    toTime = time.mktime(datetime.datetime.strptime(toTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
    return fromTime, toTime

def unixTime_windows(element):
    currentStart = element[:31]
    currentEnd = element[32:63]
    # Conver to Unix time
    currentStart = time.mktime(datetime.datetime.strptime(currentStart, "%I:%M:%S%p on %B %d, %Y").timetuple())
    currentEnd = time.mktime(datetime.datetime.strptime(currentEnd, "%I:%M:%S%p on %B %d, %Y").timetuple())
    return currentStart, currentEnd

def buildIntervalObject(dataList):
    intervals = []
    for line in dataList:
        start, end = unixTime_intervals(line)
        intervals.append(Interval(start, end))
    return intervals

def buildWindowObject(dataList):
    intervals = []
    for i in range(len(dataList)):
        if intervals == [] or dataList[i] != dataList[i-1]:
            start, end = unixTime_windows(dataList[i])
            intervals.append(Interval(start, end))
    return intervals

def lastingTimeRatio(appName):
    # Get interval template's time
    raw_intervals = pd.read_csv("interval_template.csv")
    raw_intervals = pd.DataFrame(raw_intervals)
    # package to Interval object
    intervals = buildIntervalObject(raw_intervals.time_intervals)
    
    # clean up windows log
    windowContent = [line.strip() for line in open(LOG_DIR + "windowAudit.log", 'r')]
    windowContent = [line for line in windowContent if line != "" and "---5min Window Audit milestone" not in line and appName in line]
    # package to Interval object
    appBucket = buildWindowObject(windowContent)


    # Two pointers
    i, j = 0, 0
    appLastingTime_ratio = []
    current = appBucket[0]
    inv = intervals[0]
    
    while j < len(appBucket) and i < len(intervals):
        inv = intervals[i]
        if current.start > inv.end:
            i += 1
            appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))
        else:
            if current.end <= inv.end:
                j += 1
                inv.lasting += (current.end - current.start)
                if j < len(appBucket):
                    current = appBucket[j]
            else:
                inv.lasting += (min(inv.end, current.end) - current.start)
                current.start = inv.end
                i += 1
                appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))

    # Traverse the rest time intervals
    while i < len(intervals):
        inv = intervals[i]
        appLastingTime_ratio.append(inv.lasting * 1.0 / (inv.end - inv.start))
        i += 1
    
    df = pd.DataFrame(appLastingTime_ratio, columns=[appName + '_count'])
    df.to_csv(appName + "_count_feature.csv", index=False, sep=' ')
    return df



In [36]:
lastingTimeRatio("Google Chrome")[:5]

,Google Chrome_count
0,0.820598
1,0.723333
2,0.926910
3,0.990000
4,1.000000


In [37]:
lastingTimeRatio("WeChat")[:5]

,WeChat_count
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [38]:
lastingTimeRatio("iTerm2")[:5]

,iTerm2_count
0,0.023256
1,0.006667
2,0.006645
3,0.000000
4,0.000000


# Fetch Chrome Tab Count in 5 mins

In [39]:
def chromeTabCount():
    chromeTabCountContent = [line.strip() for line in open(LOG_DIR + "chromeTabCount.log", 'r')]
    chromeTabCountContent = [line for line in chromeTabCountContent if line != "" and "---5min Chrome Tab Count Audit milestone" not in line]
    df = pd.DataFrame(chromeTabCountContent, columns=['chrome_tab_count'])
    df.to_csv("chrome_tab_count.csv", index=False, sep=' ')
    return df

len(chromeTabCount())

43

# Fetch Chrome Tab's activity in 5 mins

In [40]:
def chromeTabOpen():
    chromeTabActContent = [line.strip() for line in open(LOG_DIR + "chromeTab.log", 'r')]
    openAct = []
    start = True
    count = 0
    for line in chromeTabActContent:
        if "---5min Chrome Tab Audit milestone" in line:
            # Ignore the first line
            if start:
                start = False
            else:
                openAct.append(count)
                count = 0
        elif "open" in line:
            count += 1
    openAct.append(count)
        
    df = pd.DataFrame(openAct, columns=['chrome_tab_open_act'])
    df.to_csv("chrome_tab_open_feature.csv", index=False, sep=' ')
    return df

chromeTabOpen()[:5]

,chrome_tab_open_act
0,16
1,0
2,3
3,1
4,0


In [41]:
def chromeTabClose():
    chromeTabActContent = [line.strip() for line in open(LOG_DIR + "chromeTab.log", 'r')]
    openAct = []
    start = True
    count = 0
    for line in chromeTabActContent:
        if "---5min Chrome Tab Audit milestone" in line:
            # Ignore the first line
            if start:
                start = False
            else:
                openAct.append(count)
                count = 0
        elif "close" in line:
            count += 1
    openAct.append(count)
        
    df = pd.DataFrame(openAct, columns=['chrome_tab_close_act'])
    df.to_csv("chrome_tab_close_feature.csv", index=False, sep=' ')
    return df

chromeTabClose()[:5]

,chrome_tab_close_act
0,10
1,0
2,2
3,0
4,0


# 5 mins filter

In [42]:
# Concatenate features
def concatenate(fileList, outputFileName):
    import pandas as pd

    dfs = []
    for filename in fileList:
        # read the csv, making sure the first two columns are str
        df = pd.read_csv(filename, header=None, converters={0: str, 1: str})
        # throw away all but the first two columns
        df = df.ix[:,:1]
        # change the column names so they won't collide during concatenation
        df.columns = [filename + str(cname) for cname in df.columns]
        dfs.append(df)

    # concatenate them horizontally
    merged = pd.concat(dfs,axis=1)
    # write it out
    merged.to_csv(outputFileName, header=None, index=None)
    return merged


concatenate(["interval_template.csv", "chrome_tab_close_feature.csv", "chrome_tab_count.csv", "chrome_tab_open_feature.csv", "cmd_number_monitor.csv", "cmd_tab_combo_monitor.csv", "decoy_file_monitor.csv", "doc_file_monitor.csv", "Google Chrome_count_feature.csv", "img_file_monitor.csv", "iTerm2_count_feature.csv", "port_closed_monitor.csv", "port_open_monitor.csv", "process_deleted_monitor.csv","process_open_monitor.csv", "typing_speed_monitor.csv", "WeChat_count_feature.csv", "windows_touched_feature.csv"], "finalizedData.csv")[:5]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


,interval_template.csv0,chrome_tab_close_feature.csv0,chrome_tab_count.csv0,chrome_tab_open_feature.csv0,cmd_number_monitor.csv0,cmd_tab_combo_monitor.csv0,decoy_file_monitor.csv0,doc_file_monitor.csv0,Google Chrome_count_feature.csv0,img_file_monitor.csv0,iTerm2_count_feature.csv0,port_closed_monitor.csv0,port_open_monitor.csv0,process_deleted_monitor.csv0,process_open_monitor.csv0,typing_speed_monitor.csv0,WeChat_count_feature.csv0,windows_touched_feature.csv0
0,time_intervals,chrome_tab_close_act,chrome_tab_count,chrome_tab_open_act,Num of cmd,Num of cmd tab combo,Num of decoy touched,Num of doc touched,Google Chrome_count,Num of img touched,iTerm2_count,Num of port closed,Num of port opened,Num of process deleted,Num of process opened,Average Typing Speed,WeChat_count,touched_windows_count
1,2018-12-03-13:10:53--2018-12-03-13:15:54,10,9,16,0,0,0,0,0.8205980066445183,0,0.023255813953488372,-1,-1,-1,-1,0.0,0.0,6
2,2018-12-03-13:15:54--2018-12-03-13:20:54,0,6,0,18,0,0,0,0.7233333333333334,0,0.006666666666666667,4,3,13,7,1.55,0.0,3
3,2018-12-03-13:20:54--2018-12-03-13:25:55,2,6,3,20,1,0,0,0.9269102990033222,0,0.006644518272425249,3,8,5,3,2.21,0.0,6
4,2018-12-03-13:25:55--2018-12-03-13:30:55,0,7,1,20,0,0,0,0.99,0,0.0,8,37,5,9,2.1,0.0,2


In [43]:
def filter(inputFile, outputFile):
    """
    Only Storing those intervals roughly within 5 mins
    """
    import time
    import datetime
    def unixTime_intervals(element):
        fromTime, toTime = element[:40].split("--")
        # Conver to unix time
        fromTime = time.mktime(datetime.datetime.strptime(fromTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
        toTime = time.mktime(datetime.datetime.strptime(toTime, "%Y-%m-%d-%H:%M:%S").timetuple()) 
        return fromTime, toTime
    
    inputFile = [line.strip() for line in open(inputFile, 'r')]
    for line in inputFile[1:]:
        fromTime, toTime = unixTime_intervals(line)
        if (toTime - fromTime) >= 300 and (toTime - fromTime) <= 305:
            f = open(outputFile, "a+")
            f.write(line+"\n")
            f.close() 
    return

# filter("features_intermediate.csv", "final_fetures_file.csv")

In [44]:
filter("./finalizedData.csv", "./cleanedFinalizedData.csv")